In [1]:
import codecs
from itertools import cycle

## Set 1: Basics

### CHALLENGE 1: Convert hex to base64

In [2]:
def hextob64(hexstr):
  b64 = codecs.encode(codecs.decode(hexstr, 'hex'),'base64')
  return b64

In [3]:
hexstring = '49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d'
bsixtyfour = hextob64(hexstring)
print(bsixtyfour.decode())
print(codecs.decode(bsixtyfour,'base64'))

SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t

b"I'm killing your brain like a poisonous mushroom"


### CHALLENGE 2: Fixed XOR

In [4]:
def fixed_xor(A, B, kind='bytes'):
  if kind == 'hex' or kind == 'base64' or kind == 'utf-8':
    A = codecs.decode(A, kind) 
    B = codecs.decode(B, kind)
  out = bytes([a ^ b for (a,b) in zip(A, B)])
  return out

In [5]:
s1 = '1c0111001f010100061a024b53535009181c'
s2 = '686974207468652062756c6c277320657965'
ans = '746865206b696420646f6e277420706c6179'
fxor = fixed_xor(s1, s2, 'hex')
print(fxor)
print(fxor.hex())

b"the kid don't play"
746865206b696420646f6e277420706c6179


### CHALLENGE 3: Single-byte XOR cipher

In [6]:
def score_phrase(phrase):
  # relative frequency of letters in English language
  freqstr = 'zqjxkvbpgyfwmculdrhsnioate'  
  freq = dict(zip(cycle(freqstr), range(len(freqstr))))
  score = 0
  for char in phrase:
    if char in freq:
      score += freq[char]
  return score

In [7]:
def find_cipher(text):
  cipher_keys = [bytes([i]*len(text)) for i in range(256)]
  if type(text) == bytes:
    textbytes = text
  else:
    textbytes = codecs.decode(text, 'hex')
  scores = {}
  outputs = {}
  for ck in cipher_keys:
    sol = fixed_xor(textbytes, ck).decode(errors='ignore')
    scores[ck[0]] = score_phrase(sol)
    outputs[ck[0]] = sol
  #max_keys = [key for key, value in scores.items() if value == max(scores.values())]
  #solutions = {k: outputs[k] for k in max_keys}
  max_value = max(scores, key=scores.get)
  best = {max_value: outputs[max_value]}
  return best

In [8]:
s3 = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'

sols = find_cipher(s3)
for s in sols:
  print(s, sols[s])

88 Cooking MC's like a pound of bacon


### CHALLENGE 4: Detect single-character XOR

In [9]:
with open('4.txt') as f:
  contents = f.readlines()

lines = []
bests = []
for c in contents:
  lines.append(find_cipher(c.strip()))

scores = {}
for solutions in lines:
  for s in solutions:
    scores[solutions[s]] = score_phrase(solutions[s])

best = max(scores, key=scores.get)
best_solution = [key for key, value in scores.items() if value == max(scores.values())]

for m in best_solution:
  print(m)
  

Now that the party is jumping



### CHALLENGE 5: Implement repeating-key XOR

In [10]:
def key_xor(text, k):
  if type(text) == str:
    textbytes = text.encode()
  else:
    textbytes = text
  cipher_key = k.encode()*(int(len(text)/len(k))+1)
  return fixed_xor(textbytes, cipher_key)

In [11]:
challenge_phrase = "Burning 'em, if you ain't quick and nimble\nI go crazy when I hear a cymbal"
kx = key_xor(challenge_phrase, "ICE")
print(kx.hex())

0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f


### CHALLENGE 6: Break repeating-key XOR

In [12]:
def hamming_distance(str1, str2):
  if type(str1) == str:
    str1 = str1.encode()
  if type(str2) == str:
    str2 = str2.encode()
  n = 0
  for (x,y) in zip(str1, str2):
    n = n + bin(x^y).count('1')
  return n

In [13]:
test1 = "this is a test"
test2 = "wokka wokka!!!"

print(hamming_distance(test1, test2))

37


In [14]:
def find_keysize(text):
  distances = {}
  for ks in range(2, 40):
    dist = 0
    length = int(len(text)/ks-1)
    for i in range(length):
      dist = dist + hamming_distance(text[ks*i:ks*(i+1)], text[ks*(i+1):ks*(i+2)])
    distance = dist/(length*ks)
    distances[ks] = distance
  best = min(distances, key=distances.get)
  return best

In [15]:
def find_repeating_key(text, keysize):
  key_blocks = [[] for _ in range(keysize)]
  for i, bbyte in enumerate(text):
    key_blocks[i % keysize].append(bbyte)

  newkeys = []
  for kb in key_blocks:
    newkeys.append(find_cipher(bytes(kb)))

  keystr = ""
  for nk in newkeys:
    keystr += chr(list(nk.keys())[0])
  return keystr

In [16]:
with open('5.txt') as f:
  contents = f.read()
contents = codecs.decode(contents.encode(), 'base64')

keysize = find_keysize(contents)
keystr = find_repeating_key(contents, keysize)
print(keystr)

plaintext = key_xor(contents, keystr).decode()
print(plaintext)


Terminator X: Bring the noise
I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take 

### CHALLENGE 7: AES in ECB mode

In [17]:
from cryptography.hazmat.primitives.ciphers import ( Cipher, algorithms, modes )
from cryptography.hazmat.primitives import padding

def encrypt_ecb(plaintext, key):
  encryptor = Cipher(algorithms.AES(key), modes.ECB(),).encryptor()
  return encryptor.update(plaintext)+encryptor.finalize()

def decrypt_ecb(ciphertext, key):
  decryptor = Cipher(algorithms.AES(key), modes.ECB(),).decryptor()
  return decryptor.update(ciphertext)+decryptor.finalize()


In [18]:
aeskey = 'YELLOW SUBMARINE'
with open('7.TXT') as f:
  ct = f.read()
ct = codecs.decode(ct.encode(), 'base64')

msg7 = decrypt_ecb(ct, aeskey.encode())
print(msg7.decode())

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


### CHALLENGE 8: Detect AES in ECB mode

In [19]:
def count_repetitions(ciphertext, block_size):
  repeating_blocks = 0
  size = int(len(ciphertext)/block_size)
  blocks = [ciphertext[i*block_size:(i+1)*block_size] for i in range(size)]
  repeating_blocks = len(blocks) - len(set(blocks))

  return repeating_blocks

In [20]:
with open('8.txt') as f:
  ct = f.readlines()

for c in ct:
  b = c.strip()
  reps = count_repetitions(b, 16)
  if reps > 0:
    print(reps, " repetitions found in ", b)

6  repetitions found in  d880619740a8a19b7840a8a31c810a3d08649af70dc06f4fd5d2d69c744cd283e2dd052f6b641dbf9d11b0348542bb5708649af70dc06f4fd5d2d69c744cd2839475c9dfdbc1d46597949d9c7e82bf5a08649af70dc06f4fd5d2d69c744cd28397a93eab8d6aecd566489154789a6b0308649af70dc06f4fd5d2d69c744cd283d403180c98c8f6db1f2a3f9c4040deb0ab51b29933f2c123c58386b06fba186a


## Set 2: Block Crypto

### CHALLENGE 9: Implement PCKS#7 padding

In [21]:
def pad_plaintext(plaintext, block_size):
  if type(plaintext) == str:
    plaintext = plaintext.encode()
  padsize = (block_size - len(plaintext) % block_size)%block_size
  paddedtext = plaintext+bytes([padsize]*padsize)
  return paddedtext

In [22]:
pt = "YELLOW SUBMARINE"
pt_padded = pad_plaintext(pt, 20)
pt_padded

b'YELLOW SUBMARINE\x04\x04\x04\x04'

### CHALLENGE 10: Implement CBC mode

In [23]:
def encrypt_cbc(plaintext, key, iv, block_size):
  if type(plaintext) == str:
    plaintext = plaintext.encode()
  pt_padded = pad_plaintext(plaintext, block_size)
  num_blocks = int(len(pt_padded)/block_size)
  ct = b''
  blocks = [pt_padded[i*block_size:(i+1)*block_size] for i in range(num_blocks)]
  for block in blocks:
    blockxor = fixed_xor(block, iv)
    iv = encrypt_ecb(blockxor, key)
    ct += iv
  return ct

def decrypt_cbc(ciphertext, key, iv, block_size):
  num_blocks = int(len(ciphertext)/block_size)
  pt = b''
  blocks = [ciphertext[i*block_size:(i+1)*block_size] for i in range(num_blocks)]
  for block in blocks:
    blockdec = decrypt_ecb(block, key)
    pt += fixed_xor(blockdec, iv)
    iv = block
  return pt

In [24]:
aeskey = 'YELLOW SUBMARINE'
teststr = 'THIS IS JUST A TEST OF CBC'
testiv = b'\x00'*16
testct = encrypt_cbc(teststr, aeskey.encode(), testiv, 16)
print(testct)
testpt = decrypt_cbc(testct, aeskey.encode(), testiv, 16)
print(testpt)


b'\xd6u\xf2<dR\x1aM\x12PS\xf1\x94\xd5\xe5\xbc\xfc\xe20\xc6`\x17\xdchZ=J\xd2\x90\xc4)\x8e'
b'THIS IS JUST A TEST OF CBC\x06\x06\x06\x06\x06\x06'


In [25]:
with open('10.txt') as f:
  ct = f.read()

ct = codecs.decode(ct.encode(), 'base64')
pt = decrypt_cbc(ct, aeskey.encode(), testiv, 16)
print(pt.decode())

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


### CHALLENGE 11: An ECB/CBC detection oracle

In [26]:
import random
def get_random_aes_key(num_bytes):
  return bytes([random.randint(0, 255) for _ in range(num_bytes)])

def encryption_oracle(data):
  block_size = 16
  rand_key = get_random_aes_key(block_size)
  rand_iv = get_random_aes_key(block_size)
  pt = get_random_aes_key(random.randint(5,10)) + data
  pt += get_random_aes_key(random.randint(5,10))
  pt = pad_plaintext(pt, block_size)
  if random.randint(0,1) == 1:
    mode = 'ECB'
    ct = encrypt_ecb(pt, rand_key)
  else:
    mode = 'CBC'
    ct = encrypt_cbc(pt, rand_key, rand_iv, block_size)
  return ct, mode

def detect_encryption_mode(ciphertext, block_size):
  reps = count_repetitions(ciphertext, block_size)
  if reps > 0:
    return 'ECB'
  else:
    return 'CBC'

In [27]:
testdata = b'just some random data to encrypt '*1000

errors = 0
for _ in range(1000):
  ct, mode = encryption_oracle(testdata)
  detected_mode = detect_encryption_mode(ct, 16)
  if mode != detected_mode:
    errors += 1

print(errors, " errors in mode prediction")

0  errors in mode prediction


### CHALLENGE 12: Byte-at-a-time ECB decryption (Simple)

In [28]:
def AES_128_ECB(plaintext, random_key):
  block_size = 16
  pt = pad_plaintext(plaintext, block_size)
  ct = encrypt_ecb(pt, random_key)
  return ct

In [29]:
def break_ECB_one_byte(plaintext, key):
  bsize = 0
  last_length = len(AES_128_ECB(plaintext, key))
  for i in range(1,65):
    pt = b'A'*i + plaintext
    ciphertext = AES_128_ECB(pt, key)
    bsize = len(ciphertext) - last_length 
    if bsize > 0:
      break

  mode = detect_encryption_mode(b'A'*bsize*2, bsize)
  print(bsize)
  print(mode)

  input_block = b'A'*(bsize-1)
  bytestr = b''

  for b in range(len(plaintext)):
    outct = AES_128_ECB(input_block+plaintext[b:], key)
    ctblock = outct[:bsize]

    ctbytes = {}
    for i in range(0,256):
      block = input_block+i.to_bytes(1, 'big')
      ctbytes[block] = AES_128_ECB(block, key)

    for key, val in ctbytes.items():
      if val == ctblock:
        bytestr += key[-1:]
  return bytestr

In [30]:
with open('12.txt') as f:
  pt = f.read()
pt = codecs.decode(pt.encode(), 'base64')
randaeskey = get_random_aes_key(16)

answer = break_ECB_one_byte(pt, randaeskey)
print(answer)

16
ECB
b"Rollin' in my 5.0\nWith my rag-top down so my hair can blow\nThe girlies on standby waving just to say hi\nDid you stop? No, I just drove by\n"


### CHALLENGE 13: ECB cut-and-paste

In [31]:
def parse_profile(text):
  return dict(vals.split('=') for vals in text.split('&'))

def unpad_plaintext(text):
  pad = int.from_bytes(text[-1:], 'big')
  for b in text[-pad:]:
    if b != pad:
      return text
  return text[:-pad]

In [32]:
cookiestr = 'foo=bar&baz=qux&zap=zazzle'
strdict = parse_profile(cookiestr)
strdict

{'foo': 'bar', 'baz': 'qux', 'zap': 'zazzle'}

In [33]:
def profile_for(email):
  email = str(email).replace('&','').replace('=','')
  return "email="+email+"&uid=10&role=user"

In [34]:
print(profile_for('test=em=&ail@google.com'))

email=testemail@google.com&uid=10&role=user


In [35]:
def encrypt_profile(profile, key):
  prof = pad_plaintext(profile, 16)
  return encrypt_ecb(prof, key)

def decrypt_profile(ciphertext, key):
  prof = decrypt_ecb(ciphertext, key)
  prof = unpad_plaintext(prof)
  return parse_profile(prof.decode())

In [36]:
profaeskey = get_random_aes_key(16)
admin_email = 'email@ggl.com'   # this will be the email we make an admin. 
    # It is sized such that the last block begins after '...&role=', so the last block can be replaced with 'admin'
fake_email = 'tenchars..'       # this places 'admin' in the second encrypted block 
    # after making the first 16 byte block = 'email=tenchars..'
# Pad the second block as if it were the last block
padded_fake_email = fake_email.encode()+pad_plaintext(b'admin', 16)  

fake_profile = profile_for(padded_fake_email.decode())
fake_encrypted_profile = encrypt_profile(fake_profile, profaeskey)

admin_profile = profile_for(admin_email)
admin_encrypted_profile = encrypt_profile(admin_profile, profaeskey)

# cut second encrypted block from fake email and make it last encrypted block of desired admin email
new_admin_profile = admin_encrypted_profile[:-16] + fake_encrypted_profile[16:32]   
admin_decrypted_profile = decrypt_profile(new_admin_profile, profaeskey)

print(admin_profile)
print(admin_decrypted_profile)

email=email@ggl.com&uid=10&role=user
{'email': 'email@ggl.com', 'uid': '10', 'role': 'admin'}


### CHALLENGE 14: Byte-at-a-time ECB decryption (Harder)

In [37]:
random_prefix = get_random_aes_key(random.randint(1,15))
def break_ECB_one_byte_prefix(plaintext, key):
  bsize = 0
  prefix_size = 0
  last_length = len(AES_128_ECB(random_prefix, key))
  for i in range(1,65):
    pt = b'A'*i
    ciphertext = AES_128_ECB(random_prefix + pt, key)
    bsize = len(ciphertext) - last_length 
    if bsize > 0:
      prefix_size = bsize - i + 1
      break

  mode = detect_encryption_mode(b'A'*bsize*2, bsize)
  print('prefix size:', prefix_size)
  print('block size: ', bsize)
  print(mode)

  input_block = b'A'*(bsize-prefix_size-1)
  bytestr = b''

  for b in range(len(plaintext)):
    outct = AES_128_ECB(random_prefix + input_block + plaintext[b:], key)
    ctblock = outct[:bsize]

    ctbytes = {}
    for i in range(0,256):
      block = random_prefix+input_block+i.to_bytes(1, 'big')
      ctbytes[block] = AES_128_ECB(block, key)

    for key, val in ctbytes.items():
      if val == ctblock:
        bytestr += key[-1:]
  return bytestr

In [38]:
with open('12.txt') as f:
  pt = f.read()
pt = codecs.decode(pt.encode(), 'base64')
randaeskey = get_random_aes_key(16)

result = break_ECB_one_byte_prefix(pt, randaeskey) 
print(result)

prefix size: 7
block size:  16
ECB
b"Rollin' in my 5.0\nWith my rag-top down so my hair can blow\nThe girlies on standby waving just to say hi\nDid you stop? No, I just drove by\n"


### CHALLENGE 15: PCKS#7 padding violation

In [39]:
# same as previous unpad function but throws an error as instructed by challenge
def unpad_plaintext_pkcs7(text):     
  pad = int.from_bytes(text[-1:], 'big')
  for b in text[-pad:]:
    if b != pad:
      raise Exception('Not a valid PKCS7 padding')
  return text[:-pad]

In [40]:
valid_str = "ICE ICE BABY\x04\x04\x04\x04"
invalid_str = "ICE ICE BABY\x05\x05\x05\x05"

unpadded_str = unpad_plaintext_pkcs7(valid_str.encode())
unpadded_str1 = unpad_plaintext_pkcs7(invalid_str.encode())

Exception: Not a valid PKCS7 padding

### CHALLENGE 16: CBC bitflipping attacks

In [41]:
def encrypt_arb_str_cbc(plaintext, key, iv):
  pt = "comment1=cooking%20MCs;userdata=" 
  pt += plaintext.replace(';', '";"').replace('=','"="') 
  pt += ";comment2=%20like%20a%20pound%20of%20bacon"
  print("String to be encrypted: ", pt)
  return encrypt_cbc(pt, key, iv, 16)

def find_admin_str(ciphertext, key, iv):
  plaintext = unpad_plaintext_pkcs7(decrypt_cbc(ciphertext, key, iv, 16))
  return (b';admin=true;' in plaintext, plaintext)

In [42]:
randkey = get_random_aes_key(16)
randiv = get_random_aes_key(16)

dummystr = "YELLOW SUBMARINE"
adminstr = b";data;admin=true"

userdata = dummystr + fixed_xor(adminstr, b'A'*16).decode()

cbc_cipher = encrypt_arb_str_cbc(userdata, randkey, randiv)
cbc_cipher = cbc_cipher[0:32] + fixed_xor(cbc_cipher[32:48], b'A'*16) + cbc_cipher[48:]

adminfound, adminpt = find_admin_str(cbc_cipher, randkey, randiv)
print("Phrase ';admin=true;' found? ", adminfound)
print("Decrypted input string: ", adminpt)


String to be encrypted:  comment1=cooking%20MCs;userdata=YELLOW SUBMARINEz% 5 z %,(/|534$;comment2=%20like%20a%20pound%20of%20bacon
Phrase ';admin=true;' found?  True
Decrypted input string:  b'comment1=cooking%20MCs;userdata=U\x14\x94`~\x0f~\x8c>\xedH\xaal6[\x8b;data;admin=true;comment2=%20like%20a%20pound%20of%20bacon'
